# UNData API Exercise


In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs (Application Programming Interface) instead of downloading csv files.


In [1]:
import requests
import pandas as pd
import numpy as np
import json


You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).


1. Use the API to get all available data for the _GDP per capita, PPP (constant 2021 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [2]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'
params = {
    'format': 'json',
    'Per_page' : 30000
}
response = requests.get(endpoint, params=params)
len(response.json()[1])

17024

In [3]:
res = (
    pd
    .json_normalize(response.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'indicator.id',
            'indicator.value'})
    .rename(
        columns={
            'date':'year',
            'value':'gdp_per_capita',
            'country.id':'country_id',
            'country.value':'country_or_region'})
    .loc[:,['year', 'country_id', 'country_or_region', 'gdp_per_capita']]
)
print(res.shape)
res.head(5)


(17024, 4)


,year,country_id,country_or_region,gdp_per_capita
0,2023,ZH,Africa Eastern and Southern,4047.007031
1,2022,ZH,Africa Eastern and Southern,4038.638689
2,2021,ZH,Africa Eastern and Southern,3994.171654
3,2020,ZH,Africa Eastern and Southern,3919.499230
4,2019,ZH,Africa Eastern and Southern,4130.057222



2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.


In [4]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'
params = {
    'format' : 'json',
    'Per_page': 30000
}
response2 = requests.get(endpoint, params=params)
len(response2.json()[1])

17024

In [5]:
res2 = (
    pd
    .json_normalize(response2.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'indicator.id',
            'indicator.value'})
    .rename(
        columns={
            'date':'year',
            'value':'life_expectancy',
            'country.id':'country_id',
            'country.value':'country_or_region'})
    .loc[:,['year', 'country_id', 'country_or_region', 'life_expectancy']]
)
print(res2.shape)
res2.head(5)

(17024, 4)


,year,country_id,country_or_region,life_expectancy
0,2023,ZH,Africa Eastern and Southern,NaN
1,2022,ZH,Africa Eastern and Southern,62.899031
2,2021,ZH,Africa Eastern and Southern,62.454590
3,2020,ZH,Africa Eastern and Southern,63.313860
4,2019,ZH,Africa Eastern and Southern,63.755678



3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.


In [6]:
res_merged =(
    pd.merge(
        left=res,
        right=res2,
        on=['year', 'country_id', 'country_or_region'],
        how='outer')
)
print(res_merged.shape)
res_merged

(17024, 5)


,year,country_id,country_or_region,gdp_per_capita,life_expectancy
0,1960,1A,Arab World,NaN,44.972899
1,1960,1W,World,NaN,50.894331
2,1960,4E,East Asia & Pacific (excluding high income),NaN,37.754070
3,1960,7E,Europe & Central Asia (excluding high income),NaN,61.309416
4,1960,8S,South Asia,NaN,45.102367
...,...,...,...,...,...
17019,2023,ZJ,Latin America & Caribbean,19458.858264,NaN
17020,2023,ZM,Zambia,3718.700564,NaN
17021,2023,ZQ,Middle East & North Africa,18887.409085,NaN
17022,2023,ZT,IDA & IBRD total,13997.880812,NaN



4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.


In [7]:
endpoint='https://api.worldbank.org/v2/country/all'
params={
    'format':'json',
    'Per_page':30000
}
response3 = requests.get(endpoint, params=params)
len(response3.json()[1])

296

In [8]:
res3 = (
    pd
    .json_normalize(response3.json()[1])
    .rename(columns={'name':'country_or_region'})
)
res3
res_merged2 = (
    pd
    .merge(left=res3, right=res_merged, on='country_or_region', how='left')
    .replace(r'^\s*$', np.nan, regex=True).dropna(subset='capitalCity')
    .drop(
        columns=[
            'id',
            'iso2Code',
            'region.id',
            'region.iso2code',
            'region.value',
            'adminregion.id',
            'incomeLevel.iso2code',
            'lendingType.id',
            'lendingType.iso2code',
            'lendingType.value'])
    .loc[:, ['year',
             'country_or_region',
             'capitalCity',
             'longitude',
             'latitude',
             'incomeLevel.value',
             'gdp_per_capita',
             'life_expectancy']]
    .sort_values(['country_or_region', 'year'])
    .reset_index()
)
print(res_merged2.shape)
display(res_merged2.head())
len(res_merged2['country_or_region'].unique())

(13504, 9)


,index,year,country_or_region,capitalCity,longitude,latitude,incomeLevel.value,gdp_per_capita,life_expectancy
0,128,1960,Afghanistan,Kabul,69.1761,34.5228,Low income,NaN,32.535
1,129,1961,Afghanistan,Kabul,69.1761,34.5228,Low income,NaN,33.068
2,130,1962,Afghanistan,Kabul,69.1761,34.5228,Low income,NaN,33.547
3,131,1963,Afghanistan,Kabul,69.1761,34.5228,Low income,NaN,34.016
4,132,1964,Afghanistan,Kabul,69.1761,34.5228,Low income,NaN,34.494


211

## **Bonus Questions:** 

After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.



1. Adjust your request so that it returns data just for the United States.


`attempt at importing each indicator separately for an easier merge later on.`

In [9]:
# request world bank api data for USA indicator NY.GDP.PCAP.PP.KD - GDP per capita
endpoint='https://api.worldbank.org/v2/country/us/indicator/NY.GDP.PCAP.PP.KD'
params={
    'format':'json',
    'Per_page':30000
}
response4 = requests.get(endpoint, params=params)
# len(response4.json()[1])

In [10]:
# request world bank api data for USA indicator SP.DYN.LE00.IN - life expectancy
endpoint='https://api.worldbank.org/v2/country/us/indicator/SP.DYN.LE00.IN'
params={
    'format':'json',
    'Per_page':30000
}
response5 = requests.get(endpoint, params=params)
# len(response5.json()[1])

In [11]:
# create dataframe out of .json and clean up
res4 = (
    pd.
    json_normalize(response4.json()[1])
    .drop(columns={'unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value'})
    .rename(columns={'value':'gdp_per_capita'})
)
# print(res4.shape)
# res4.head(3)

In [12]:
# create dataframe out of .json and clean up
res5 = (
    pd.
    json_normalize(response5.json()[1])
    .drop(columns={'unit', 'obs_status', 'decimal','indicator.id', 'indicator.value'})
    .rename(columns={'value':'life_expectancy'})
)
# print(res5.shape)
# res5.head(3)

In [13]:
# merge the two dfs together so it's USA data with both categories
res_merged_3 = (
    pd.merge(left=res4,
             right=res5,
             on=['countryiso3code',
                 'date',
                 'country.id',
                 'country.value'],
                 how='outer')
        .drop(columns=['country.id', 'countryiso3code'])
        .rename(columns={'date':'year', 'country.value':'country'})
        .sort_values(['year'])
)
res_merged3 = res_merged_3.loc[:,['year', 'country', 'gdp_per_capita', 'life_expectancy']]
# display(res_merged3)

`i'll try to import both indicators at the same time.`

In [14]:
# world bank api request for multiple indicators, needs a source call at the end if multiple indicators
endpoint='https://api.worldbank.org/v2/country/us/indicator/NY.GDP.PCAP.PP.KD;SP.DYN.LE00.IN?source=2'
params={
    'format':'json',
    'Per_page':'296'
}
response6 = requests.get(endpoint, params=params)
# print(len(response6.json()[1]))
res6 = (
    pd
    .json_normalize(response6.json()[1])
    .drop(columns={'scale', 'unit', 'obs_status','decimal'})
)
display(res6.head(2))
display(res6.tail(2))



,countryiso3code,date,value,indicator.id,indicator.value,country.id,country.value
0,USA,2023,73637.302789,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",US,United States
1,USA,2022,72165.483470,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",US,United States


,countryiso3code,date,value,indicator.id,indicator.value,country.id,country.value
126,USA,1961,70.270732,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States
127,USA,1960,69.770732,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",US,United States


In [15]:
# function to find the index where the df changes from GDPPC to LE
def find_first_different_string(df, column_name):
    """Finds the first string in a DataFrame column that
    is different from the first value."""
    
    for value in df[column_name]:
        if value != df[column_name].iloc[0]:
            return df.index[df[column_name]==value][0]
    return None, None

In [16]:
# use function to define our break point for this df
change_index = find_first_different_string(res6, 'indicator.value')
# slice the GDP portion out of larger df
gdp_half = (
    res6
    .iloc[0:change_index]
    .sort_values('date')
    .drop(
        columns=[
            'countryiso3code',
            'indicator.id',
            'indicator.value',
            'country.id'])
    .rename(
        columns={
            'value':'gdp_per_capita',
            'country.value':'country'})
    .copy()
)
# slice the life expectancy portion out of larger df
life_exp_half = (
    res6
    .iloc[change_index:]
    .sort_values('date')
    .drop(
        columns=[
            'countryiso3code',
            'indicator.id',
            'indicator.value',
            'country.id'])
    .rename(
            columns={
                'value':'life_expectancy',
                'country.value':'country'})
    .copy()
)
# merge the two halves so that useful information in its own columns
us_data = (
    pd
    .merge(
        left=gdp_half,
        right=life_exp_half,
        on=['date', 'country'],
        how='outer')
    .loc[:,['date','country', 'gdp_per_capita', 'life_expectancy']]
)
us_data

,date,country,gdp_per_capita,life_expectancy
0,1960,United States,NaN,69.770732
1,1961,United States,NaN,70.270732
2,1962,United States,NaN,70.119512
3,1963,United States,NaN,69.917073
4,1964,United States,NaN,70.165854
...,...,...,...,...
59,2019,United States,69458.610183,78.787805
60,2020,United States,67266.187874,76.980488
61,2021,United States,71055.876194,76.329268
62,2022,United States,72165.483470,77.434146



2. Adjust your request so that it returns data just for the United States for the year 2021.


In [17]:
# world bank api request for multiple indicators, needs a source call at the end if multiple indicators
endpoint='https://api.worldbank.org/v2/country/us/indicator/NY.GDP.PCAP.PP.KD;SP.DYN.LE00.IN'#?source=2'
params={
    'format':'json',
    'Per_page': 30000,
    'date':'2021',
    'source': 2
}
response7 = requests.get(endpoint, params=params)
response7.json()[1]
res7 = (
    pd
    .json_normalize(response7.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'country.id',
            'indicator.id'})
    .loc[:,['date', 'indicator.value', 'country.value', 'value']]
)
display(res7)


,date,indicator.value,country.value,value
0,2021,"GDP per capita, PPP (constant 2021 internation...",United States,71055.876194
1,2021,"Life expectancy at birth, total (years)",United States,76.329268



3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.


In [26]:
# world bank api request for multiple indicators, needs a source call at the end if multiple indicators
endpoint='https://api.worldbank.org/v2/country/us/indicator/NY.GDP.PCAP.PP.KD;SP.DYN.LE00.IN'
params={
    'format':'json',
    'Per_page': 30000,
    'date':'2000:2021',
    'source': 2
}
response8 = requests.get(endpoint, params=params)
response8.json()[1]
res8 = (
    pd
    .json_normalize(response8.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'country.id',
            'indicator.id'})
    .loc[:,['date', 'indicator.value', 'country.value', 'value']]
)
# display(res8)
change_index = find_first_different_string(res8, 'indicator.value')
gdp_half = (
    res8
    .iloc[0:change_index]
    .sort_values('date')
    # .drop(
    #     columns=[
    #         'countryiso3code',
    #         'indicator.id',
    #         'indicator.value',
    #         'country.id'])
    .rename(
        columns={
            'value':'gdp_per_capita',
            'country.value':'country'})
    .copy()
)

# slice the life expectancy portion out of larger df
life_exp_half = (
    res8
    .iloc[change_index:]
    .sort_values('date')
    # .drop(
    #     columns=[
    #         'countryiso3code',
    #         'indicator.id',
    #         'indicator.value',
    #         'country.id'])
    .rename(
            columns={
                'value':'life_expectancy',
                'country.value':'country'})
    .copy()
)
# merge the two halves so that useful information in its own columns
us_data_2 = (
    pd
    .merge(
        left=gdp_half,
        right=life_exp_half,
        on=['date', 'country'],
        how='outer')
    .loc[:,['date','country', 'gdp_per_capita', 'life_expectancy']]
)
us_data_2

,date,country,gdp_per_capita,life_expectancy
0,2000,United States,55059.219169,76.636585
1,2001,United States,55037.893673,76.836585
2,2002,United States,55456.862403,76.936585
3,2003,United States,56519.350712,77.036585
4,2004,United States,58153.387828,77.487805
5,2005,United States,59627.059056,77.487805
6,2006,United States,60699.272085,77.687805
7,2007,United States,61329.539227,77.987805
8,2008,United States,60821.185961,78.039024
9,2009,United States,58736.946133,78.390244



4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.


In [30]:
# world bank api request for multiple indicators, needs a source call at the end if multiple indicators
endpoint='https://api.worldbank.org/v2/country/us;ca/indicator/NY.GDP.PCAP.PP.KD;SP.DYN.LE00.IN'
params={
    'format':'json',
    'Per_page': 30000,
    'date':'2000:2021',
    'source': 2
}
response9 = requests.get(endpoint, params=params)
response9.json()[1]
res9 = (
    pd
    .json_normalize(response9.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'country.id',
            'indicator.id'})
    .loc[:,['date', 'indicator.value', 'country.value', 'value']]
)
# display(res8)
change_index = find_first_different_string(res9, 'indicator.value')
gdp_half = (
    res9
    .iloc[0:change_index]
    .sort_values('date')
    # .drop(
    #     columns=[
    #         'countryiso3code',
    #         'indicator.id',
    #         'indicator.value',
    #         'country.id'])
    .rename(
        columns={
            'value':'gdp_per_capita',
            'country.value':'country'})
    .copy()
)

# slice the life expectancy portion out of larger df
life_exp_half = (
    res9
    .iloc[change_index:]
    .sort_values('date')
    # .drop(
    #     columns=[
    #         'countryiso3code',
    #         'indicator.id',
    #         'indicator.value',
    #         'country.id'])
    .rename(
            columns={
                'value':'life_expectancy',
                'country.value':'country'})
    .copy()
)
# merge the two halves so that useful information in its own columns
us_data_3 = (
    pd
    .merge(
        left=gdp_half,
        right=life_exp_half,
        on=['date', 'country'],
        how='outer')
    .loc[:,['date','country', 'gdp_per_capita', 'life_expectancy']]
    .sort_values(['country','date'])
)
us_data_3

,date,country,gdp_per_capita,life_expectancy
0,2000,Canada,47667.433086,79.166829
2,2001,Canada,48036.626618,79.398049
4,2002,Canada,48943.541241,79.530732
6,2003,Canada,49381.595036,79.715122
8,2004,Canada,50436.294843,79.992195
10,2005,Canada,51564.845565,80.112683
12,2006,Canada,52391.379262,80.543659
14,2007,Canada,52948.899427,80.543659
16,2008,Canada,52899.475450,80.724634
18,2009,Canada,50772.869922,81.066098



5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.



6. The endpoint "<http://api.worldbank.org/v2/indicator>" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.